In [1]:
from ultralytics import YOLO
import cv2
from facenet_pytorch import InceptionResnetV1, MTCNN
from services.embeddings import extract_embeddings, compare_embeddings
from services.utils import face_detection
from PIL import Image, ExifTags
import torch
from torchvision import transforms
import numpy as np

d:\Visium\vas-backend\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
resnet = InceptionResnetV1(pretrained='casia-webface').eval()
preprocess = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5]),
])

def get_embeddings(face):
    if isinstance(face, np.ndarray):
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = Image.fromarray(face)
    
    face_tensor = preprocess(face)
    face_tensor = face_tensor.unsqueeze(0)
    
    with torch.no_grad():
        embeddings = resnet(face_tensor)
    
    return embeddings

In [3]:
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, img = cap.read()
    
    if ret == True:
        count+=1
    else:
        continue
    
    if count == 30:
        break
    
cap.release()

In [4]:
model = YOLO("Models/faceDetection/face_detection.pt")

results = model.predict(img, conf=0.4)

results = results[0]

bboxes = face_detection(results)

for box in bboxes:
    x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
    face = img[int(y1):int(y2), int(x1):int(x2)]
    emb = get_embeddings(face)


0: 480x640 1 face, 162.8ms
Speed: 4.7ms preprocess, 162.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


In [5]:
img = cv2.imread("reference2.jpg")

results = model.predict(img, conf=0.4)

results = results[0]

bboxes = face_detection(results)
print(bboxes)
for box in bboxes:
    x1, y1, x2, y2 = box[0], box[1], box[2], box[3]
    face = img[int(y1):int(y2), int(x1):int(x2)]
    emb2 = get_embeddings(face)


0: 384x640 1 face, 128.3ms
Speed: 4.0ms preprocess, 128.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
[[547.21728515625, 106.11264038085938, 743.463134765625, 368.1576232910156]]


In [6]:
print(compare_embeddings(emb, emb2))

False
